In [28]:
import spacy
from spacy import displacy



import spacy_stanza

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.tag import pos_tag
import re


nlp = spacy.load('en_core_web_sm')
nltk.download('punkt')

import stanza

stanza.download('en')
nlpStanza = stanza.Pipeline(lang='en', processors='tokenize,ner,mwt,pos,lemma,depparse')

# Initialize the pipeline
nlpSS = spacy_stanza.load_pipeline("en")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kolvi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-09-17 20:01:10 INFO: Downloading default packages for language: en (English) ...
2023-09-17 20:01:10 INFO: File exists: C:\Users\kolvi\stanza_resources\en\default.zip
2023-09-17 20:01:13 INFO: Finished downloading models and saved to C:\Users\kolvi\stanza_resources.
2023-09-17 20:01:13 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-09-17 20:01:14 WARNING: Can not find mwt: default from official model list. Ignoring it.
2023-09-17 20:01:15 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| ner       | ontonotes |

2023-09-17

In [29]:
import graphviz
from graphviz import Source

class plotDependencyGraph:
    def __init__(self, name = ""):
        self.subgraph = []
        self.name = name
        self.mainGraph = []
        

    # shall be moved to specific function, here just for convinience and debugging
    # def addSubgraph(self, name, dependency, parserType):
    #     if parserType == "stanza":
    #         head, rel, dependent = dependency
    #         self.addNode(name, head.text, dependent.text, rel)
        
    #     elif parserType == "spacy":
    #         head, dependent, rel = dependency
    #         self.addNode(name, head, dependent, rel)

    def addSubgraph(self, name, head, dependent, rel):
        g = []
        g = graphviz.Digraph(comment= name)
        g.node(head)
        g.node(dependent)
        g.edge(head, dependent, label = rel)
        self.subgraph.append(g)

    def createTree(self):
        self.mainGraph = graphviz.Digraph(comment= self.name)
        for graph in self.subgraph:
            self.mainGraph.subgraph(graph)

        # Render the graph to a PNG file.
        self.mainGraph.render(self.name, view=True, format="png")

In [30]:
# removePointIndices = r"^[0-9a-zA-Z]+\s*[-\\.)}]?\s+"
usePattern = r"[^a-zA-Z0-9\-\_\[\]\(\)\{\}\s]+"
tokenizer = RegexpTokenizer(usePattern)

def preprocess(corpus):
    #get sentences
    sentences = sent_tokenize(corpus)

    # prepare sentences
    # for i, sentence in enumerate(sentences):
    #     sentences[i] = word_tokenize(sentence) #re.sub(usePattern, "", sentence).lower().strip()
    
    return sentences

In [31]:
def preprocessdict(corpus):
    #get sentences
    sentences = [p for p in corpus.split('\n') if p]    
    return sentences

In [32]:
# def remove_repeated_words(string):
#     words = string.split()
#     unique_words = []
    
#     for word in words:
#         if word not in unique_words:
#             unique_words.append(word)
    
#     return ' '.join(unique_words)

In [33]:
# from collections import defaultdict
# import re

# functionType = r"<(.*?)>"

# class StackStorage:
#     def __init__(self, key = "", relation = "", pos = 0):
#         self.key = key
#         self.relation = relation
#         self.pos = pos
        

#     def getValues(self):
#         return self.key, self.relation, self.pos


    
# class RuleBasedExtractor:
#     def __init__(self, relations = defaultdict(), words = defaultdict(), rules =[]):
#         # current key of the dict
#         self.currentWordKey = ""
#         # currect relation type
#         self.currentRelationType = ""
#         # current position of the relation in the current word key dict
#         self.currentPositionOfRelIndict = 0
#         # maximum position of the List
#         self.currentWordKeyMaxRelations = 0


#         # depth of key level stored
#         # store depth and postion when moving to next position like a stack
#         self.depthStack = []
#         # depth of key level Integer
#         self.depthStackLevel = 0
        
       

#         # original relations
#         self.words = words
#         #original relations
#         self.relations = relations

#         # rules
#         self.rules = rules

#         # This indexing is needed when we go deeoer in the stack
#         # current exceuting key 
#         self.currentExeKey = ""
#         # current exceuting relation 
#         self.currentExeRelation = ""
#         # current exe postions
#         self.currentExePostion = 0

#         # predicted output string for each significant word key
#         self.outputValue = ""
#         # predicted relationship type for each string
#         self.relationValue = ""


#         # flag proceed current execution of certain rule sets due to a failure
#         self.proceed = True
#         self.checkIfCond = False


#         #phrases
#         self.phrases = defaultdict(set)
        

#     def printPhrases(self):
#         for p in self.phrases.keys():
#             print(p, [val for val in self.phrases[p]], sep="\n")


#     def execute(self):
#         # loop throught the relation keys
#         for key in self.relations.keys():
#             # store current key
#             self.currentWordKey = key
#             # get values of the key
#             values = self.relations[key]
#             # get the max relations assoiated with the key
#             self.currentWordKeyMaxRelations = len(values)

#             for i, value in enumerate(values):
#                 # extra layer of error condition
#                 # extract relation with respect to columns of individual relations
#                 if i < self.currentWordKeyMaxRelations:
#                     self.currentRelationType = value
#                     self.currentPositionOfRelIndict = i
#                     self.ruleParser()
#                     self.addPhraseRelations()

#             # add all modifers in the tree row
#             #self.addModifiers()
#             self.addPhraseRelations()

#     def addPhraseRelations(self):
#         # self.outputValue += self.currentExeKey
#         if self.outputValue != "":
#             #remove some repeated words
#             output = remove_repeated_words(self.outputValue)
#             self.phrases[self.currentExeKey].add(output)
#         self.outputValue = ""


#     def getValue(self, key, relation, pos):
#         if self.relations[key][pos] == relation:
#             return self.words[key][pos]
        
#     def returnToRoot(self):
#         for i, val in enumerate(self.depthStack):
#             self.levelup()
#             # print("Leveled up")

#     def ruleParser(self):
#         # update current Executing reference
#         self.setCurrentExeStates(self.currentWordKey, self.currentRelationType, self.currentPositionOfRelIndict)

#         # state machine to extract some infomation based on some rules
#         for line in self.rules:
#             if self.stripL(line).startswith("<"):
#                 self.function = re.findall(functionType, line)[0]
#                 self.extractType = self.stripL(line[line.find('@')+1:].strip().strip('\t'))
#                 #call the function and set the flag if the extraction needs to be performed or not
#                 self.functionCalls()

#     def functionCalls(self):
#         # start of any major rule
#         if self.function == "if":
#             self.proceed = True
#             self.returnToRoot()
#             self.checkIfCond = self.when()
#             return self.checkIfCond
        
#         # returns of a relation is part of the main key in a relation
#         elif self.function == "findIf" and self.checkIfCond:
#             self.proceed = self.findIf()
#             return self.proceed

#         # go down in the tree in a particular condition
#         elif self.function == "leveldown" and self.checkIfCond:
#             self.proceed = self.leveldown()
#             return self.proceed

#         # go up in the tree in a particular condition
#         elif self.function == "levelup" and self.checkIfCond:
#            return self.levelup()

#         #go down the tree untill the last element to pick all components of a relation
#         elif self.function == "addVerticalForAll" and self.checkIfCond and self.proceed:
#             self.loopUntillVerticalTree()
#             return self.proceed
        
#         #insert head if necessary to a derived Relation to current output
#         elif self.function == "insert" and self.checkIfCond and self.proceed:
#             self.insertHead()
#             return self.proceed
        
#         #add modifiers if necessary
#         elif self.function == "addModifiers" and self.checkIfCond and self.proceed:
#             self.addModifiers()

#         # add relation
#         elif self.function == "add" and self.checkIfCond and self.proceed:
#             self.addCurrentRelation()

#         else:
#             return False


#     def stripL(self, line):
#         return line.strip().strip('\t')

            
#     def when(self):
#         if self.extractType == self.currentExeRelation:
#             return True
#         else:
#             return False

#     def findIf(self):
#         #check if relation type is present update the key, level and postion
#         # for now only detects first instance
#         # multiple instances must be handled
#         if self.extractType in self.relations[self.currentExeKey] and self.proceed:
#             self.setCurrentExeStates(self.currentExeKey, self.extractType, self.relations[self.currentExeKey].index(self.extractType))
#             return True
#         else:
#             return False
              
#     def levelup(self):
#         if self.depthStackLevel > 0:
#             # reinstate the old states
#             self.degradeToOldState()
#             self.depthStack.pop()
#             self.depthStackLevel -= 1
#             return True
#         else:
#             return False


#     def leveldown(self):
#         if self.currentExeKey in self.words.keys():
#             # go to new state
#             self.depthStack.append(StackStorage(self.currentExeKey, self.currentExeRelation, self.currentExePostion))
#             self.depthStackLevel += 1

#             # try to update to new state
#             flag = self.updateToNewState()

#             # in case update failed then revert to lower state back
#             if flag == False:
#                 self.levelup()
#                 return False
            
#             return True
#         else:
#             return False
        
#     def loopUntillVerticalTree(self):
#         # to avoid coming back to same state scenario
#         statesVisited = []
#         ret = self.leveldown()
#         # loop untill unknown depth and stop where you can find last conpound in vertical tree
#         while(ret):
#             if ret and self.findIf() and self.currentExeKey not in statesVisited:
#                 statesVisited.append(self.currentExeKey)
#                 ret = self.leveldown()
#                 continue
#             else:
#                 self.levelup()
#                 break
			

#         # add those compounds and its modifiers if any
#         while(self.depthStackLevel):
#             self.addToOutput(self.getValue(self.currentExeKey, self.extractType, self.currentExePostion))
#             #self.addModifiers()
#             self.levelup()
        
#         # put back the extractType to currentExeRelation type
#         self.extractType = self.currentExeRelation

#         # function always return True
#         # can be used when you want to add new values for validation
#         return True
    

#     def insertHead(self):
#         self.addToOutput(self.currentExeKey)
#         # function always return True
#         # can be used when you want to add new values for validation
#         return True
    
#     def addCurrentRelation(self):
#         if self.extractType == self.extractType:
#             self.addToOutput(self.getValue(self.currentExeKey, self.extractType, self.currentExePostion))



#     # add modifiers if necessary
#     def addModifiers(self):
#         # update extract type to previous extract type
#         self.extractType = self.currentExeRelation
#         for i, rel in enumerate(self.relations[self.currentExeKey]):
#             if rel == "advmod" or rel == "nummod" or rel == "acl:relcl" :
#                 self.addToOutput(self.getValue(self.currentExeKey, rel, i))
#             # if rel == "amod" or rel == "appos" or rel == "nmod" or rel == "nmod:poss":
#             #     self.addToOutput(self.getValue(self.currentExeKey, rel, i))
#             if rel == "obl" or rel == "compound":
#                 self.addToOutput(self.getValue(self.currentExeKey, rel, i))
                

#         # function always return True
#         # can be used when you want to add new values for validation
#         return True        

#     def setCurrentExeStates(self, key = "", relation = "", pos = 0):
#         self.currentExeKey = key
#         self.currentExeRelation =  relation
#         self.currentExePostion = pos


#     def updateToNewState(self):
#         newKey = self.words[self.currentExeKey][self.currentExePostion]
#         if newKey in self.words.keys():
#             # relation type and postion will be checked later on depending on condition
#             self.setCurrentExeStates(newKey)
#             return True
#         else:
#             return False

#     def degradeToOldState(self):
#         key, relation, pos = self.depthStack[-1].getValues()
#         self.setCurrentExeStates(key, relation, pos)


#     def addToOutput(self, data):
#         if data != "" and data != None:
#             self.outputValue += data + " "

#     def clearOutput(self):
#         self.outputValue =  ""



In [34]:
def remove_repeated_words(string):
    unique_words = []
    
    for word in string:
        if word not in unique_words:
            unique_words.append(word)
    
    return unique_words

In [35]:
from collections import defaultdict
import re

functionType = r"<(.*?)>"

class StackStorage:
    def __init__(self, key = "", relation = "", pos = 0):
        self.key = key
        self.relation = relation
        self.pos = pos
        

    def getValues(self):
        return self.key, self.relation, self.pos


    
class RuleBasedExtractor:
    def __init__(self, relations = defaultdict(), words = defaultdict(), rules =[]):
        # current key of the dict
        self.currentWordKey = ""
        # currect relation type
        self.currentRelationType = ""
        # current position of the relation in the current word key dict
        self.currentPositionOfRelIndict = 0
        # maximum position of the List
        self.currentWordKeyMaxRelations = 0


        # depth of key level stored
        # store depth and postion when moving to next position like a stack
        self.depthStack = []
        # depth of key level Integer
        self.depthStackLevel = 0
        
       

        # original relations
        self.words = words
        #original relations
        self.relations = relations

        # rules
        self.rules = rules

        # This indexing is needed when we go deeoer in the stack
        # current exceuting key 
        self.currentExeKey = ""
        # current exceuting relation 
        self.currentExeRelation = ""
        # current exe postions
        self.currentExePostion = 0

        # predicted output string for each significant word key
        self.outputValue = []
        # predicted relationship type for each string
        self.relationValue = ""


        # flag proceed current execution of certain rule sets due to a failure
        self.proceed = True
        self.checkIfCond = False


        #phrases
        self.phrases = defaultdict(list)
        

    def printPhrases(self):
        for p in self.phrases.keys():
            print(p, [val for val in self.phrases[p]], sep="\n")


    def execute(self):
        # loop throught the relation keys
        for key in self.relations.keys():
            # store current key
            self.currentWordKey = key
            # get values of the key
            values = self.relations[key]
            # get the max relations assoiated with the key
            self.currentWordKeyMaxRelations = len(values)

            for i, value in enumerate(values):
                # extra layer of error condition
                # extract relation with respect to columns of individual relations
                if i < self.currentWordKeyMaxRelations:
                    self.currentRelationType = value
                    self.currentPositionOfRelIndict = i
                    self.ruleParser()
                    self.addPhraseRelations()

            # add all modifers in the tree row
            #self.addModifiers()
            self.addPhraseRelations()

        for p in self.phrases.keys():
            return [val for val in self.phrases[p]]


    def addPhraseRelations(self):
        # self.outputValue += self.currentExeKey
        if self.outputValue != []:
            #remove some repeated words
            output = remove_repeated_words(self.outputValue)
            self.phrases[self.currentExeKey].append(output)
            self.outputValue.clear()


    def getValue(self, key, relation, pos):
        if self.relations[key][pos] == relation:
            return self.words[key][pos]
        
    def returnToRoot(self):
        for i, val in enumerate(self.depthStack):
            self.levelupWithoutDegradation()
            self.setCurrentExeStates(self.currentWordKey, self.currentRelationType, self.currentPositionOfRelIndict)
            # print("Leveled up")

    def ruleParser(self):
        # update current Executing reference
        self.setCurrentExeStates(self.currentWordKey, self.currentRelationType, self.currentPositionOfRelIndict)

        # state machine to extract some infomation based on some rules
        for line in self.rules:
            if self.stripL(line).startswith("<"):
                self.function = re.findall(functionType, line)[0]
                self.extractType = self.stripL(line[line.find('@')+1:].strip().strip('\t'))
                #call the function and set the flag if the extraction needs to be performed or not
                self.functionCalls()

    def functionCalls(self):
        # start of any major rule
        if self.function == "if":
            self.proceed = True
            self.returnToRoot()
            self.proceed = self.when()
            pass

        
        # returns of a relation is part of the main key in a relation
        elif self.function == "findIf" and self.proceed:
            self.proceed = self.findIf()


        # go down in the tree in a particular condition
        elif self.function == "leveldown" and self.proceed:
            self.proceed = self.leveldown()


        # go up in the tree in a particular condition
        elif self.function == "levelup" and self.proceed:
           self.proceed =  self.levelup()

        #go down the tree untill the last element to pick all components of a relation
        elif self.function == "addVerticalForAll" and self.proceed:
            self.loopUntillVerticalTree()
        
        #insert head if necessary to a derived Relation to current output
        elif self.function == "insert" and self.proceed:
            self.insertHead()

        
        #add modifiers if necessary
        elif self.function == "addModifiers" and self.proceed:
            self.addModifiers()

        # add relation
        elif self.function == "add" and self.proceed:
            self.addCurrentRelation()

        else:
            return False


    def stripL(self, line):
        return line.strip().strip('\t')

            
    def when(self):
        if self.extractType == self.currentExeRelation:
            return True
        else:
            return False

    def findIf(self):
        #check if relation type is present update the key, level and postion
        # for now only detects first instance
        # multiple instances must be handled
        if self.extractType in self.relations[self.currentExeKey] and self.proceed:
            self.setCurrentExeStates(self.currentExeKey, self.extractType, self.relations[self.currentExeKey].index(self.extractType))
            return True
        else:
            return False
              
    def levelup(self):
        if self.depthStackLevel > 0:
            # reinstate the old states
            self.degradeToOldState()
            self.depthStack.pop()
            self.depthStackLevel -= 1
            return True
        else:
            return False
        
    # To level up when no proper states are present
    def levelupWithoutDegradation(self):
        while(True):
            if self.depthStackLevel > 0:
                self.depthStack.pop()
                self.depthStackLevel -= 1
            else:
                return True


        


    def leveldown(self):
        if self.currentExeKey in self.words.keys():
            # go to new state
            self.depthStack.append(StackStorage(self.currentExeKey, self.currentExeRelation, self.currentExePostion))
            self.depthStackLevel += 1

            # try to update to new state
            flag = self.updateToNewState()

            # in case update failed then revert to lower state back
            if flag == False:
                self.levelup()
                return False
            
            return True
        else:
            return False
        
    def loopUntillVerticalTree(self):
        # to avoid coming back to same state scenario
        statesVisited = []
        ret = self.leveldown()
        # loop untill unknown depth and stop where you can find last conpound in vertical tree
        while(ret):
            if ret and self.findIf() and self.currentExeKey not in statesVisited:
                statesVisited.append(self.currentExeKey)
                ret = self.leveldown()
                continue
            else:
                self.levelup()
                break
			

        # add those compounds and its modifiers if any
        while(self.depthStackLevel):
            self.addToOutput(self.getValue(self.currentExeKey, self.extractType, self.currentExePostion))
            self.addModifiers()
            self.levelup()
        
        # put back the extractType to currentExeRelation type
        self.extractType = self.currentExeRelation

        # function always return True
        # can be used when you want to add new values for validation
        return True
    

    def insertHead(self):
        self.addToOutput(self.currentExeKey)
        # function always return True
        # can be used when you want to add new values for validation
        return True
    
    def addCurrentRelation(self):
        if self.extractType == self.extractType:
            self.addToOutput(self.getValue(self.currentExeKey, self.extractType, self.currentExePostion))



    # add modifiers if necessary
    def addModifiers(self):
        # update extract type to previous extract type
        poolValues = []
        self.extractType = self.currentExeRelation
        for i, rel in enumerate(self.relations[self.currentExeKey]):
            # if rel == "advmod" or rel == "nummod" or rel == "acl:relcl" :
            #     self.addToOutput(self.getValue(self.currentExeKey, rel, i), True)
            # if rel == "amod" or rel == "appos" or rel == "nmod" or rel == "nmod:poss":
            #     self.addToOutput(self.getValue(self.currentExeKey, rel, i))
            if rel == "obl" or rel == "compound" or rel == "advmod" or rel == "amod":
                poolValues.append(self.getValue(self.currentExeKey, rel, i))

        for val in reversed(poolValues):
            self.addToOutput(val, True)
                

        # function always return True
        # can be used when you want to add new values for validation
        return True        

    def setCurrentExeStates(self, key = "", relation = "", pos = 0):
        self.currentExeKey = key
        self.currentExeRelation =  relation
        self.currentExePostion = pos


    def updateToNewState(self):
        newKey = self.words[self.currentExeKey][self.currentExePostion]
        if newKey in self.words.keys():
            # relation type and postion will be checked later on depending on condition
            self.setCurrentExeStates(newKey)
            return True
        else:
            return False

    def degradeToOldState(self):
        key, relation, pos = self.depthStack[-1].getValues()
        self.setCurrentExeStates(key, relation, pos)


    def addToOutput(self, data, addModiferToPrevWord = False):
        if data != "" and data != None:
            if addModiferToPrevWord:
                self.outputValue[-1] = data + " " + self.outputValue[-1]
            else:
                self.outputValue.append(data)

    def clearOutput(self):
        self.outputValue.clear()


In [36]:
import json

In [37]:
class TagSentencesReqT1:
    def __init__(self):
        self.tags = ["O", "B-STATE", "I-STATE", "B-VAL"]
        self.tagDict = {j:i for i,j in enumerate(self.tags)}
        
        # Specify the JSON file path
        json_file_path = "tags.json"

        # Write the dictionary to a JSON file
        with open(json_file_path, "w") as json_file:
            json.dump({i:j for i,j in enumerate(self.tags)}, json_file)
            
    # to tag words are in the form ["State", "Val", "Val"]
    # internal words are tagged as B-VAL, I-VAL
    def tag(self, doc, toTagWords):
        states = []
        values = []
        for i, sentence in enumerate(doc.sentences):
            if len(toTagWords) == 3:
                states = [token.text for token in sentence.tokens if token.text in toTagWords[0].split()]
                values = [token.text for token in sentence.tokens if token.text in toTagWords[1].split() + toTagWords[2].split()]
                return self.tagCreateSentTags(sentence, states, values)

            elif len(toTagWords) == 2:
                states = [token.text for token in sentence.tokens if token.text in toTagWords[0].split()]
                values = [token.text for token in sentence.tokens if token.text in toTagWords[1].split()]
                return  self.tagCreateSentTags(sentence, states, values)

            elif len(toTagWords) == 1:
                states = [token.text for token in sentence.tokens if token.text in toTagWords[0].split()]
                return self.tagCreateSentTags(sentence, states)

            else:
                return self.tagCreateSentTags(sentence)

    

    def tagCreateSentTags(self, sentence, states = [], values = []):
        taggedSentence = {}

        for token in sentence.tokens:
            if len(states) or len(values): 
                if token.text == states[0]:
                    taggedSentence[token.text] = self.tagDict["B-STATE"]

                elif token.text in states[1:]:
                    taggedSentence[token.text] = self.tagDict["I-STATE"]
 
                elif token.text in values:
                    taggedSentence[token.text] = self.tagDict["B-VAL"]


                else:
                    taggedSentence[token.text] = self.tagDict["O"]

            else:
                taggedSentence[token.text] = self.tagDict["O"]

        return taggedSentence
                





    

    

In [38]:
from IPython.core.display import display_html, HTML
from pathlib import Path
from collections import defaultdict

import pydot
import os

reqPath = r"C:\Users\kolvi\AppData\Local\anaconda3\envs\NLP\Library\bin"
if reqPath not in os.environ["PATH"]:
    os.environ["PATH"] += os.pathsep + reqPath
# os.environ["PATH"] += os.pathsep + 
# print(os.environ["PATH"])



def dependencyParserTokenBased(sentence,  plot = plotDependencyGraph("Trial"), rules = []):

    # stanza document
    docStanza = nlpStanza(sentence)

    words = defaultdict(list)
    relations = defaultdict(list)  
    headUpos =  defaultdict(list)

    # loop through sentences and its dependencies and add subgraph
    for sent in docStanza.sentences:
        for dependency in sent.dependencies:
            head, rel, dependent = dependency
            words[head.text].append(dependent.text)
            relations[head.text].append(rel)
            headUpos[head.text] = head.upos
            name = head.text + "_stanza_" + dependent.text
            plot.addSubgraph(name, head.text, dependent.text, rel)

    rbe = RuleBasedExtractor(relations, words, rules)
    toTagWords = rbe.execute()
    # print(f'{sentence} \n')
    # rbe.printPhrases()


    t1 = TagSentencesReqT1()
    if toTagWords != None:
        return t1.tag(docStanza, toTagWords[0])



    


In [39]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

In [40]:
def createParFiles(toks, tags, file = ""):

    allSentences = {}
    
    allSentences["tokens"] = {i:j for i, j in enumerate(toks)}
    allSentences["ner_tags"] = {i:j for i, j in enumerate(tags)}

    df = pd.DataFrame(allSentences)

    # Create a PyArrow Table from the DataFrame
    table = pa.Table.from_pandas(df)

    # Define the Parquet file path
    parquet_file_path = file + '.parquet'

    # Write the Table to a Parquet file
    pq.write_table(table, parquet_file_path)

    print(f'Data {file} with many rows has been written to {parquet_file_path}')

In [44]:
dataRules = open(r".\rulebook\dict_identy_objects.txt", "r").read()
rules = preprocessdict(dataRules)


allTokens = []
allTages = []



data = open("try1.txt", "r").read().__str__().replace("\r"," ").replace("\n"," ")
sentences = preprocess(data)

plot = plotDependencyGraph("plot")

# # testStr = "The condition that CSWS transitions from system OFF state to system ON state are as follows."

for i,sentence in enumerate(sentences):
    plot = plotDependencyGraph("plot")
    ret = dependencyParserTokenBased(sentence, plot, rules)

    if ret != None:
        allTokens.append(list(ret.keys()))
        allTages.append(list(ret.values()))
        
    # plot.createTree()

# allSentences["tokens"] = {i:j for i, j in enumerate(allTokens)}
# allSentences["ner_tags"] = {i:j for i, j in enumerate(allTages)}
TrainData  = 0

if TrainData:
    dataLen = len(allTokens)
    train = round(0.7*dataLen)
    val = round(0.1*dataLen)


    createParFiles(allTokens[:train], allTages[:train], "train")
    createParFiles(allTokens[train:train+val], allTages[train:train+val], "val")
    createParFiles(allTokens[train+val: ], allTages[train+val: ], "test")

else:
    createParFiles(allTokens, allTages, "par_test")





Data par_test with many rows has been written to par_test.parquet
